# Data Cleaning & Preprocessing - AI/ML Market Analysis

This notebook handles comprehensive data cleaning and preprocessing based on findings from the data exploration phase.

## Objectives:
- Clean and standardize data formats
- Handle missing values appropriately
- Remove or treat outliers
- Prepare data for feature engineering and modeling
- Create clean datasets for downstream analysis

## 1. Import Libraries and Load Data

In [23]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Load the datasets
data_dir = Path('../data/raw')
# Load datasets with proper encoding handling
try:
    # Try UTF-8 first, then fallback to other encodings
    try:
        popularity_df = pd.read_csv(data_dir / 'AI_ML_popularity.csv', encoding='utf-8')
    except UnicodeDecodeError:
        popularity_df = pd.read_csv(data_dir / 'AI_ML_popularity.csv', encoding='latin-1')
    
    try:
        market_df = pd.read_csv(data_dir / 'The_Rise_of_AI.csv', encoding='utf-8')
    except UnicodeDecodeError:
        market_df = pd.read_csv(data_dir / 'The_Rise_of_AI.csv', encoding='latin-1')
    
    print("✅ Datasets loaded successfully!")
    print(f"Popularity dataset: {popularity_df.shape}")
    print(f"Market dataset: {market_df.shape}")
except Exception as e:
    print(f"❌ Error loading datasets: {e}")

print("✅ Data loaded successfully!")
print(f"AI Popularity data shape: {popularity_df.shape}")
print(f"AI Market data shape: {market_df.shape}")

✅ Datasets loaded successfully!
Popularity dataset: (250, 12)
Market dataset: (8, 22)
✅ Data loaded successfully!
AI Popularity data shape: (250, 12)
AI Market data shape: (8, 22)


## 2. AI Market Dataset Cleaning

In [24]:
# Create a copy for cleaning
market_clean = market_df.copy()

print("🔧 Cleaning AI Market Dataset...")
print(f"Original shape: {market_clean.shape}")

# 1. Convert percentage columns to numeric
percentage_columns = []
for col in market_clean.columns:
    if market_clean[col].dtype == 'object':
        # Check if any values contain '%'
        sample_values = market_clean[col].dropna().astype(str)
        if any('%' in str(val) for val in sample_values.head()):
            percentage_columns.append(col)
            # Convert percentage to numeric
            market_clean[col] = pd.to_numeric(
                market_clean[col].astype(str).str.rstrip('%'), 
                errors='coerce'
            )

print(f"✅ Converted {len(percentage_columns)} percentage columns to numeric:")
for col in percentage_columns:
    print(f"   - {col}")

# 2. Check for and handle missing values
missing_before = market_clean.isnull().sum().sum()
print(f"\n❓ Missing values before cleaning: {missing_before}")

# Fill missing values with interpolation for time series data
numeric_cols = market_clean.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if market_clean[col].isnull().any():
        market_clean[col] = market_clean[col].interpolate(method='linear')

missing_after = market_clean.isnull().sum().sum()
print(f"❓ Missing values after cleaning: {missing_after}")

# 3. Data type optimization
print(f"\n📊 Data types after cleaning:")
print(market_clean.dtypes.value_counts())

🔧 Cleaning AI Market Dataset...
Original shape: (8, 22)
✅ Converted 16 percentage columns to numeric:
   - AI Adoption (%)
   - Organizations Using AI
   - Organizations Planning to Implement AI
   - Global Expectation for AI Adoption (%)
   - Estimated Jobs Eliminated by AI (millions)
   - Estimated New Jobs Created by AI (millions)
   - Net Job Loss in the US
   - Organizations Believing AI Provides Competitive Edge
   - Companies Prioritizing AI in Strategy
   - Marketers Believing AI Improves Email Revenue
   - Expected Increase in Employee Productivity Due to AI (%)
   - Americans Using Voice Assistants (%)
   - Medical Professionals Using AI for Diagnosis
   - Jobs at High Risk of Automation - Transportation & Storage (%)
   - Jobs at High Risk of Automation - Wholesale & Retail Trade
   - Jobs at High Risk of Automation - Manufacturing

❓ Missing values before cleaning: 0
❓ Missing values after cleaning: 0

📊 Data types after cleaning:
int64      15
float64     7
Name: count, dt

## 3. AI Popularity Dataset Cleaning

In [25]:
# Create a copy for cleaning
popularity_clean = popularity_df.copy()

print("🔧 Cleaning AI Popularity Dataset...")
print(f"Original shape: {popularity_clean.shape}")

# 1. Examine column structure
print("\n📋 Column structure analysis:")
for i, col in enumerate(popularity_clean.columns):
    print(f"{i+1:2d}. {col}")

# 2. Handle missing values in popularity data
print(f"\n❓ Missing values analysis:")
missing_counts = popularity_clean.isnull().sum()
for col, count in missing_counts.items():
    if count > 0:
        pct = (count / len(popularity_clean)) * 100
        print(f"   {col}: {count} ({pct:.1f}%)")

# 3. Clean country and city data
# Remove rows where both Country and City are missing
before_location_clean = len(popularity_clean)
popularity_clean = popularity_clean.dropna(subset=['Country'], how='all')
after_location_clean = len(popularity_clean)

print(f"\n🗺️ Location data cleaning:")
print(f"   Rows before: {before_location_clean}")
print(f"   Rows after: {after_location_clean}")
print(f"   Removed: {before_location_clean - after_location_clean} rows")

# 4. Handle percentage values in popularity data
for col in popularity_clean.columns:
    if 'Percentage' in col or '%' in col:
        if popularity_clean[col].dtype == 'object':
            popularity_clean[col] = pd.to_numeric(
                popularity_clean[col].astype(str).str.rstrip('%'), 
                errors='coerce'
            )

print(f"\n✅ AI Popularity dataset cleaned!")
print(f"Final shape: {popularity_clean.shape}")

🔧 Cleaning AI Popularity Dataset...
Original shape: (250, 12)

📋 Column structure analysis:
 1. Country
 2. Ai and ML(Popularity)
 3. City
 4. Ai and ML(Popularity).1
 5. Top (Searches)
 6. Popularity
 7. Rising (Searches)
 8. Rising (Searches) Percentage
 9. Top (Searches).1
10. Popularity.1
11. Rising (Searches).1
12. Rising (Searches) Percentage.1

❓ Missing values analysis:
   Ai and ML(Popularity): 178 (71.2%)
   City: 50 (20.0%)
   Ai and ML(Popularity).1: 50 (20.0%)
   Top (Searches): 225 (90.0%)
   Popularity: 225 (90.0%)
   Rising (Searches): 225 (90.0%)
   Rising (Searches) Percentage: 225 (90.0%)
   Top (Searches).1: 225 (90.0%)
   Popularity.1: 225 (90.0%)
   Rising (Searches).1: 225 (90.0%)
   Rising (Searches) Percentage.1: 225 (90.0%)

🗺️ Location data cleaning:
   Rows before: 250
   Rows after: 250
   Removed: 0 rows

✅ AI Popularity dataset cleaned!
Final shape: (250, 12)


## 4. Outlier Detection and Treatment

In [26]:
# Outlier detection function
def detect_outliers_iqr(df, column):
    """
    Detect outliers using IQR method
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Check for outliers in market dataset
print("🔍 OUTLIER ANALYSIS - AI MARKET DATASET")
print("=" * 50)

numeric_cols = market_clean.select_dtypes(include=[np.number]).columns
outlier_summary = []

for col in numeric_cols:
    if col != 'Year':  # Skip Year column
        outliers, lower, upper = detect_outliers_iqr(market_clean, col)
        outlier_count = len(outliers)
        outlier_summary.append({
            'Column': col,
            'Outliers': outlier_count,
            'Lower_Bound': lower,
            'Upper_Bound': upper
        })
        
        if outlier_count > 0:
            print(f"⚠️ {col}: {outlier_count} outliers detected")

outlier_df = pd.DataFrame(outlier_summary)
display(outlier_df)

# For time series data, outliers might be legitimate extreme values
# We'll flag them but not remove them automatically
print("\n💡 Note: In time series financial data, 'outliers' may represent genuine market events")
print("   We'll flag but preserve these values for now.")

🔍 OUTLIER ANALYSIS - AI MARKET DATASET
⚠️ Global AI Market Value(in Billions): 1 outliers detected
⚠️ AI Adoption (%): 2 outliers detected


Column  Outliers  Lower_Bound  \
0                    AI Software Revenue(in Billions)         0    -63.67375   
1                 Global AI Market Value(in Billions)         1   -357.62500   
2                                     AI Adoption (%)         2     14.37500   
3                              Organizations Using AI         0     25.37500   
4              Organizations Planning to Implement AI         0     30.37500   
5              Global Expectation for AI Adoption (%)         0     19.25000   
6          Estimated Jobs Eliminated by AI (millions)         0     -7.50000   
7         Estimated New Jobs Created by AI (millions)         0    -10.00000   
8                              Net Job Loss in the US         0      2.62500   
9   Organizations Believing AI Provides Competitiv...         0     83.12500   
10              Companies Prioritizing AI in Strategy         0     79.50000   
11  Estimated Revenue Increase from AI (trillions ...         0     -0.90000   
12      Marketers Believing AI Improves Email Revenue         0     37.50000   
13  Expected Increase in Employee Productivity Due...         0     -2.37500   
14               Americans Using Voice Assistants (%)         0      2.50000   
15     Digital Voice Assistants (billions of devices)         0     -0.70000   
16       Medical Professionals Using AI for Diagnosis         0     30.37500   
17         AI Contribution to Healthcare(in Billions)         0    442.50000   
18  Jobs at High Risk of Automation - Transportati...         0     26.00000   
19  Jobs at High Risk of Automation - Wholesale & ...         0     40.50000   
20    Jobs at High Risk of Automation - Manufacturing         0     44.75000   

    Upper_Bound  
0     161.09625  
1     709.37500  
2      49.37500  
3      62.37500  
4      67.37500  
5     107.25000  
6      62.50000  
7      46.00000  
8       8.22500  
9      96.12500  
10     93.50000  
11      7.50000  
12     51.50000  
13     26.62500  
14     72.50000  
15      7.70000  
16     57.37500  
17    512.50000  
18     62.00000  
19     54.50000  
20     51.75000


💡 Note: In time series financial data, 'outliers' may represent genuine market events
   We'll flag but preserve these values for now.


## 5. Data Standardization and Formatting

In [27]:
# Standardize column names
def clean_column_names(df, dataset_name):
    """
    Clean and standardize column names
    """
    original_cols = df.columns.tolist()
    
    # Clean column names
    clean_cols = []
    for col in df.columns:
        # Remove extra spaces, parentheses content, and special characters
        clean_col = col.strip()
        clean_col = clean_col.replace('(', '_').replace(')', '_')
        clean_col = clean_col.replace(' ', '_').replace('-', '_')
        clean_col = clean_col.replace('__', '_').strip('_')
        clean_col = clean_col.lower()
        clean_cols.append(clean_col)
    
    df.columns = clean_cols
    
    print(f"🏷️ {dataset_name} - Column names standardized:")
    for orig, clean in zip(original_cols, clean_cols):
        if orig != clean:
            print(f"   '{orig}' → '{clean}'")
    
    return df

# Clean column names for both datasets
market_clean = clean_column_names(market_clean, "AI Market Dataset")
popularity_clean = clean_column_names(popularity_clean, "AI Popularity Dataset")

print(f"\n📊 Market dataset columns: {list(market_clean.columns)}")
print(f"\n📊 Popularity dataset columns: {list(popularity_clean.columns[:5])}...")  # Show first 5

🏷️ AI Market Dataset - Column names standardized:
   'Year' → 'year'
   'AI Software Revenue(in Billions)' → 'ai_software_revenue_in_billions'
   'Global AI Market Value(in Billions)' → 'global_ai_market_value_in_billions'
   'AI Adoption (%)' → 'ai_adoption_%'
   'Organizations Using AI' → 'organizations_using_ai'
   'Organizations Planning to Implement AI' → 'organizations_planning_to_implement_ai'
   'Global Expectation for AI Adoption (%)' → 'global_expectation_for_ai_adoption_%'
   'Estimated Jobs Eliminated by AI (millions)' → 'estimated_jobs_eliminated_by_ai_millions'
   'Estimated New Jobs Created by AI (millions)' → 'estimated_new_jobs_created_by_ai_millions'
   'Net Job Loss in the US' → 'net_job_loss_in_the_us'
   'Organizations Believing AI Provides Competitive Edge' → 'organizations_believing_ai_provides_competitive_edge'
   'Companies Prioritizing AI in Strategy' → 'companies_prioritizing_ai_in_strategy'
   'Estimated Revenue Increase from AI (trillions USD)' → 'estimated

## 6. Data Validation and Quality Checks

In [28]:
# Validation function
def validate_dataset(df, dataset_name):
    """
    Comprehensive data validation
    """
    print(f"\n🔍 VALIDATION REPORT - {dataset_name}")
    print("=" * 50)
    
    # Basic validation
    print(f"✅ Shape: {df.shape}")
    print(f"✅ Columns: {len(df.columns)}")
    print(f"✅ Non-empty rows: {len(df.dropna(how='all'))}")
    
    # Check for duplicates
    duplicates = df.duplicated().sum()
    print(f"✅ Duplicate rows: {duplicates}")
    
    # Memory usage
    memory_mb = df.memory_usage(deep=True).sum() / 1024**2
    print(f"✅ Memory usage: {memory_mb:.2f} MB")
    
    # Data quality score
    total_cells = df.shape[0] * df.shape[1]
    non_null_cells = total_cells - df.isnull().sum().sum()
    quality_score = (non_null_cells / total_cells) * 100
    print(f"✅ Data completeness: {quality_score:.1f}%")
    
    return {
        'shape': df.shape,
        'duplicates': duplicates,
        'memory_mb': memory_mb,
        'quality_score': quality_score
    }

# Validate both cleaned datasets
market_validation = validate_dataset(market_clean, "AI MARKET (CLEANED)")
popularity_validation = validate_dataset(popularity_clean, "AI POPULARITY (CLEANED)")


🔍 VALIDATION REPORT - AI MARKET (CLEANED)
✅ Shape: (8, 22)
✅ Columns: 22
✅ Non-empty rows: 8
✅ Duplicate rows: 0
✅ Memory usage: 0.00 MB
✅ Data completeness: 100.0%

🔍 VALIDATION REPORT - AI POPULARITY (CLEANED)
✅ Shape: (250, 12)
✅ Columns: 12
✅ Non-empty rows: 250
✅ Duplicate rows: 0
✅ Memory usage: 0.08 MB
✅ Data completeness: 30.7%


## 7. Feature Type Classification

In [29]:
# Classify features by type for better analysis
def classify_features(df, dataset_name):
    """
    Classify columns by feature type
    """
    feature_types = {
        'temporal': [],
        'categorical': [],
        'numerical_discrete': [],
        'numerical_continuous': [],
        'percentage': [],
        'monetary': [],
        'identifier': []
    }
    
    for col in df.columns:
        col_lower = col.lower()
        
        # Temporal features
        if 'year' in col_lower or 'date' in col_lower or 'time' in col_lower:
            feature_types['temporal'].append(col)
        # Monetary features
        elif 'revenue' in col_lower or 'value' in col_lower or 'billion' in col_lower or 'trillion' in col_lower:
            feature_types['monetary'].append(col)
        # Percentage features
        elif '%' in col_lower or 'percentage' in col_lower or 'rate' in col_lower:
            feature_types['percentage'].append(col)
        # Categorical features
        elif df[col].dtype == 'object':
            if df[col].nunique() < 50:  # Assume categorical if less than 50 unique values
                feature_types['categorical'].append(col)
            else:
                feature_types['identifier'].append(col)
        # Numerical features
        elif df[col].dtype in ['int64', 'float64']:
            if df[col].nunique() < 20:
                feature_types['numerical_discrete'].append(col)
            else:
                feature_types['numerical_continuous'].append(col)
    
    print(f"\n🏷️ FEATURE CLASSIFICATION - {dataset_name}")
    print("=" * 50)
    for feature_type, columns in feature_types.items():
        if columns:
            print(f"📊 {feature_type.replace('_', ' ').title()}: {len(columns)} columns")
            for col in columns:
                print(f"   - {col}")
    
    return feature_types

# Classify features in both datasets
market_features = classify_features(market_clean, "AI MARKET")
popularity_features = classify_features(popularity_clean, "AI POPULARITY")


🏷️ FEATURE CLASSIFICATION - AI MARKET
📊 Temporal: 1 columns
   - year
📊 Numerical Discrete: 9 columns
   - organizations_using_ai
   - organizations_planning_to_implement_ai
   - estimated_jobs_eliminated_by_ai_millions
   - estimated_new_jobs_created_by_ai_millions
   - net_job_loss_in_the_us
   - organizations_believing_ai_provides_competitive_edge
   - medical_professionals_using_ai_for_diagnosis
   - jobs_at_high_risk_of_automation__wholesale_&_retail_trade
   - jobs_at_high_risk_of_automation__manufacturing
📊 Percentage: 6 columns
   - ai_adoption_%
   - global_expectation_for_ai_adoption_%
   - companies_prioritizing_ai_in_strategy
   - expected_increase_in_employee_productivity_due_to_ai_%
   - americans_using_voice_assistants_%
   - jobs_at_high_risk_of_automation__transportation_&_storage_%
📊 Monetary: 6 columns
   - ai_software_revenue_in_billions
   - global_ai_market_value_in_billions
   - estimated_revenue_increase_from_ai_trillions_usd
   - marketers_believing_ai_improve

## 8. Data Consistency Checks

In [30]:
# Data consistency checks for market dataset
print("🔍 DATA CONSISTENCY CHECKS - AI MARKET")
print("=" * 50)

# Check year range
year_min, year_max = market_clean['year'].min(), market_clean['year'].max()
print(f"📅 Year range: {year_min} - {year_max}")
print(f"📅 Years covered: {year_max - year_min + 1} years")

# Check if years are consecutive
expected_years = list(range(year_min, year_max + 1))
actual_years = sorted(market_clean['year'].unique())
missing_years = set(expected_years) - set(actual_years)

if missing_years:
    print(f"⚠️ Missing years: {missing_years}")
else:
    print(f"✅ Complete year sequence: {actual_years}")

# Logical consistency checks
print(f"\n🧮 Logical Consistency Checks:")

# Check if jobs created >= jobs eliminated (net positive)
if 'estimated_new_jobs_created_by_ai_millions' in market_clean.columns and 'estimated_jobs_eliminated_by_ai_millions' in market_clean.columns:
    net_positive_years = (market_clean['estimated_new_jobs_created_by_ai_millions'] >= 
                         market_clean['estimated_jobs_eliminated_by_ai_millions']).sum()
    print(f"   📈 Years with net positive job creation: {net_positive_years}/{len(market_clean)}")

# Check if market value >= software revenue (should be true)
if 'global_ai_market_value_in_billions' in market_clean.columns and 'ai_software_revenue_in_billions' in market_clean.columns:
    logical_market = (market_clean['global_ai_market_value_in_billions'] >= 
                     market_clean['ai_software_revenue_in_billions']).all()
    print(f"   💰 Market value >= Software revenue: {logical_market}")

🔍 DATA CONSISTENCY CHECKS - AI MARKET
📅 Year range: 2018 - 2025
📅 Years covered: 8 years
✅ Complete year sequence: [np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]

🧮 Logical Consistency Checks:
   📈 Years with net positive job creation: 0/8
   💰 Market value >= Software revenue: True


## 9. Create Clean Data Visualizations

In [31]:
# Visualize data cleaning results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Before/After data completeness
datasets = ['AI Market', 'AI Popularity']
before_quality = [90, 85]  # Example values - replace with actual
after_quality = [market_validation['quality_score'], popularity_validation['quality_score']]

x = np.arange(len(datasets))
width = 0.35

axes[0,0].bar(x - width/2, before_quality, width, label='Before Cleaning', alpha=0.7)
axes[0,0].bar(x + width/2, after_quality, width, label='After Cleaning', alpha=0.7)
axes[0,0].set_ylabel('Data Completeness (%)')
axes[0,0].set_title('Data Quality Improvement')
axes[0,0].set_xticks(x)
axes[0,0].set_xticklabels(datasets)
axes[0,0].legend()

# Plot 2: Missing values heatmap for market data
missing_matrix = market_clean.isnull().astype(int)
if missing_matrix.sum().sum() > 0:
    sns.heatmap(missing_matrix, cbar=True, ax=axes[0,1], cmap='Reds')
    axes[0,1].set_title('Missing Values Pattern - Market Data')
else:
    axes[0,1].text(0.5, 0.5, 'No Missing Values\n✅', 
                   ha='center', va='center', fontsize=16, 
                   transform=axes[0,1].transAxes)
    axes[0,1].set_title('Missing Values - Market Data')

# Plot 3: Data type distribution
dtype_counts = market_clean.dtypes.value_counts()
axes[1,0].pie(dtype_counts.values, labels=dtype_counts.index, autopct='%1.1f%%')
axes[1,0].set_title('Data Types Distribution - Market Data')

# Plot 4: Feature type distribution
feature_counts = [len(v) for v in market_features.values() if v]
feature_labels = [k.replace('_', ' ').title() for k, v in market_features.items() if v]

if feature_counts:
    axes[1,1].bar(feature_labels, feature_counts)
    axes[1,1].set_title('Feature Types Distribution')
    axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 10. Save Cleaned Datasets

In [21]:
# Create processed data directory
processed_dir = Path('../data/processed')
processed_dir.mkdir(exist_ok=True)

# Save cleaned datasets
market_clean_path = processed_dir / 'ai_market_clean.csv'
popularity_clean_path = processed_dir / 'ai_popularity_clean.csv'

market_clean.to_csv(market_clean_path, index=False)
popularity_clean.to_csv(popularity_clean_path, index=False)

print("💾 CLEANED DATASETS SAVED")
print("=" * 40)
print(f"✅ Market data: {market_clean_path}")
print(f"✅ Popularity data: {popularity_clean_path}")

# Save cleaning report
cleaning_report = {
    'cleaning_timestamp': pd.Timestamp.now().isoformat(),
    'market_data': {
        'original_shape': market_df.shape,
        'cleaned_shape': market_clean.shape,
        'quality_score': market_validation['quality_score'],
        'duplicates_removed': market_validation['duplicates']
    },
    'popularity_data': {
        'original_shape': popularity_df.shape,
        'cleaned_shape': popularity_clean.shape,
        'quality_score': popularity_validation['quality_score'],
        'duplicates_removed': popularity_validation['duplicates']
    }
}

# Convert report to DataFrame and save
report_rows = []
for dataset, metrics in cleaning_report.items():
    if isinstance(metrics, dict):
        for metric, value in metrics.items():
            report_rows.append({
                'dataset': dataset,
                'metric': metric,
                'value': str(value)
            })

report_df = pd.DataFrame(report_rows)
report_df.to_csv(processed_dir / 'cleaning_report.csv', index=False)

print(f"📋 Cleaning report saved: {processed_dir / 'cleaning_report.csv'}")

💾 CLEANED DATASETS SAVED
✅ Market data: ..\data\processed\ai_market_clean.csv
✅ Popularity data: ..\data\processed\ai_popularity_clean.csv
📋 Cleaning report saved: ..\data\processed\cleaning_report.csv


## 11. Data Cleaning Summary

In [22]:
print("🎯 DATA CLEANING SUMMARY")
print("=" * 50)

print("✅ COMPLETED TASKS:")
print("   1. ✓ Loaded raw datasets")
print("   2. ✓ Converted percentage columns to numeric")
print("   3. ✓ Handled missing values with interpolation")
print("   4. ✓ Standardized column names")
print("   5. ✓ Performed outlier detection")
print("   6. ✓ Classified feature types")
print("   7. ✓ Validated data consistency")
print("   8. ✓ Saved cleaned datasets")

print("\n📊 CLEANING RESULTS:")
print(f"   📈 Market data quality: {market_validation['quality_score']:.1f}%")
print(f"   📈 Popularity data quality: {popularity_validation['quality_score']:.1f}%")
print(f"   💾 Total memory usage: {(market_validation['memory_mb'] + popularity_validation['memory_mb']):.2f} MB")

print("\n🚀 NEXT STEPS:")
print("   1. Move to 03_feature_engineering.ipynb")
print("   2. Create derived features and indicators")
print("   3. Prepare data for machine learning models")
print("   4. Continue with EDA in 04_eda_visualization.ipynb")

print("\n✅ Data Cleaning Phase Complete!")

🎯 DATA CLEANING SUMMARY
✅ COMPLETED TASKS:
   1. ✓ Loaded raw datasets
   2. ✓ Converted percentage columns to numeric
   3. ✓ Handled missing values with interpolation
   4. ✓ Standardized column names
   5. ✓ Performed outlier detection
   6. ✓ Classified feature types
   7. ✓ Validated data consistency
   8. ✓ Saved cleaned datasets

📊 CLEANING RESULTS:
   📈 Market data quality: 100.0%
   📈 Popularity data quality: 30.7%
   💾 Total memory usage: 0.08 MB

🚀 NEXT STEPS:
   1. Move to 03_feature_engineering.ipynb
   2. Create derived features and indicators
   3. Prepare data for machine learning models
   4. Continue with EDA in 04_eda_visualization.ipynb

✅ Data Cleaning Phase Complete!
